In [ ]:
%pip install jupyter_bokeh -q
#%pip install dask-expr -q
#%pip install datashader -q

In [ ]:
import os
from io import StringIO
import json
import pandas as pd
import numpy as np
import geopandas as gpd
import rioxarray

from geodata_fetch import harvest, utils
from gis_utils.visualisation import get_geotiff_statistics

from IPython.display import display, JSON

#import holoviews as hv
import hvplot.xarray
import geoviews as gv

import panel as pn

#hvplot.extension('plotly')
#hv.extension('plotly')
#gv.extension('plotly')
#hv.extension('bokeh')
gv.extension('bokeh')
pn.extension() 

In [ ]:
# Function to list all .tif files in the specified directory
def list_tif_files(path):
    return [f for f in os.listdir(path) if f.endswith('.tif')]

# Function to load and display the selected .tif file
def load_and_display_tif(filename):
    filepath = os.path.join(data_load_path, filename)
    #img = gv.util.from_xarray(rioxarray.open_rasterio(filepath).rio.reproject('EPSG:3857'))
    img = gv.util.from_xarray(rioxarray.open_rasterio(filepath))
    
    # Define map tiles and create the map image
    map_tiles = gv.tile_sources.EsriImagery().opts(width=1000, height=600)
    map_img = gv.Image(img, kdims=['x', 'y']).opts(title=filename) #cmap='viridis', 
    map_combo = map_tiles * map_img
    
    return map_combo

In [ ]:
# notebooks/sandbox/data
geojson = geojson = {
    'body': {
        "type": "FeatureCollection",
        "name": "dissolved-boundaries",
        "crs": {
            "type": "name",
            "properties": {
                "name": "urn:ogc:def:crs:OGC:1.3:CRS84" 
            }
        },
        "features": [
            {
                "type": "Feature",
                "properties": {
                    "fid": 1
                },
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [
                        [
                            [116.26012130269045, -29.225295369642396],
                            [116.261724812149055, -29.241374854584375],
                            [116.283751968396274, -29.256813692452539],
                            [116.284342735038919, -29.268250184258388],
                            [116.292247755352392, -29.265992437426529],
                            [116.292360282331941, -29.293057573630019],
                            [116.314865678242256, -29.293523728033122],
                            [116.326259034921833, -29.293033039128805],
                            [116.326315298411629, -29.305397680579894],
                            [116.355065941687045, -29.307016748931797],
                            [116.355065941687045, -29.306575187382712],
                            [116.383366477044206, -29.307384715430175],
                            [116.384322956370426, -29.290407813444993],
                            [116.387586238777402, -29.282629879611861],
                            [116.386517232471661, -29.259807919053017],
                            [116.359201308185533, -29.259488866292969],
                            [116.359229439930417, -29.259243440415627],
                            [116.35242155766754, -29.259292525638209],
                            [116.352140240218716, -29.220237788279107],
                            [116.302234524787593, -29.223503148505326],
                            [116.281388901825679, -29.2239696200396],
                            [116.26012130269045, -29.225295369642396]
                        ]
                    ]
                }
            }
        ]
    }
}


req = geojson
geojson_data = req['body']  # Directly accessing the 'body' since it's already a dictionary in this mock setup

# Convert the GeoJSON string to a GeoDataFrame
geom = gpd.read_file(StringIO(json.dumps(geojson_data)))


gpd_lon = geom.centroid.x[0] #approximate centre latitude based on the input geojson - point roughly in the middle
gpd_lat = geom.centroid.y[0] #approximate centre longitude based on the input geojson - point roughly in the middle

# minimum rectabgle that fits the input geometry. This can be around a single polygon or multiple polygons
bbox = list(geom.total_bounds)

In [ ]:
# Path to output directory (will be created if it doesn't exist):
outpath = '/workspace/notebooks/sandbox/data/output-data/geodata-package-testing'

# Column names for latitude and longitude in input file:
colname_lat = gpd_lat
colname_lng = gpd_lon

# Bounding box: Left (min Long), Bottom (min Lat), Right (max Long), Top (max Lat)
add_buffer = False

resample=False

# Resolution of data download in arcseconds (1 arcsec ~ 30m)
target_res = 1
target_proj = "EPSG:3857"

date_start = "2022-10-01"
date_end = "2022-11-30"
# Number of time intervals to split the image collection into
time_intervals = 0

target_sources = {"DEM":"DEM",
                  "SLGA":{"Organic_Carbon": ["0-5cm"]}
                  } #                  "SLGA":{
                    #  "Organic_Carbon": ["0-5cm"]},
                  #"Radiometric":["radmap2019_grid_k_conc_filtered_awags_rad_2019"]

data_mask=True

In [ ]:
# Conversion to desired JSON format
"""
TODO: add default date inside appropriate function in harvest.py instead of here.
"""
json_data = {
    "property_name":"stardewvalley",
    "outpath": outpath,
    "data_mask": data_mask,
    "target_res": str(target_res),
    "target_crs": target_proj,
    "date_start": date_start if date_start is not None else "2022-10-01", #a date of some kind must be provided or the harvester complains
    "date_end": date_end if date_end is not None else "2022-11-30",
    "time_intervals": time_intervals,
    "target_sources": target_sources,
    "target_centroid_lat": colname_lat,
    "target_centroid_lng": colname_lng,
    "target_bbox": bbox,
    "add_buffer": add_buffer,
    "resample": resample
}

# Converting dictionary to JSON formatted string
data = json.dumps(json_data)

# Create a file-like object from JSON string
json_file_like = StringIO(data)

In [ ]:

df = harvest.run(json_file_like, geom)

Testing a generalised function to get basic pixel stats about a downloaded (and masked) geotif and return them so they can be attached as metadata:

In [ ]:
# import rasterio

# def get_geotiff_statistics(input_geotiff):

#     with rasterio.open(input_geotiff) as src:
#         data = src.read(1)  # Read the first band
#         mask = src.dataset_mask()  # Get the mask of the dataset

#         # Apply the mask
#         masked_data = np.ma.masked_array(data, mask=(mask == 0))

#         # Get statistics excluding masked pixels
#         min_val = masked_data.min()
#         max_val = masked_data.max()
#         mean_val = masked_data.mean()
#         median_val = np.ma.median(masked_data)
#         std_val = masked_data.std()

#     return {
#         "min": min_val,
#         "max": max_val,
#         "mean": mean_val,
#         "median": median_val,
#         "std": std_val,
#     }

In [11]:
geotiff_path_dem = '/workspace/notebooks/sandbox/data/output-data/geodata-package-testing/DEM_SRTM_1_Second_Hydro_Enforced_stardewvalley_masked.tif'
geotiff_path_slga = '/workspace/notebooks/sandbox/data/output-data/geodata-package-testing/SLGA_Organic_Carbon_0-5cm_stardewvalley_masked.tif'

statistics_dem = get_geotiff_statistics(geotiff_path_dem)
print(statistics_dem)

statistics_slga = get_geotiff_statistics(geotiff_path_slga)
print(statistics_slga)

INFO:botocore.credentials:Found credentials in environment variables.
{'min': 276.01416, 'max': 354.81882, 'mean': 310.1643755788422, 'median': 309.97717, 'std': 14.136959459734}
INFO:botocore.credentials:Found credentials in environment variables.
{'min': 0.51, 'max': 2.27, 'mean': 1.0193844012360074, 'median': 0.9700000286102295, 'std': 0.18882460866474265}


In [ ]:
# # testing the colour_geotiff_and_save_cog function

colormap = "magma"

for tiff in os.listdir(data_load_path):
    if tiff.endswith('_masked.tif'):  
        print(f"reading in geotiff: {tiff}")
        
        output_tiff_filename = os.path.join(data_load_path, tiff)
        utils.colour_geotiff_and_save_cog(output_tiff_filename, colormap)
    else:
        pass

In [ ]:
# # Create the dropdown menu
# data_load_path = '/workspace/notebooks/sandbox/data/output-data/geodata-package-testing'
# tif_files = list_tif_files(data_load_path)

# dropdown = pn.widgets.Select(name='Select a .tif file', options=tif_files)

# # Panel DynamicMap to update the map based on the dropdown selection
# @pn.depends(dropdown.param.value)
# def update_map(selected_file):
#     return load_and_display_tif(selected_file)

# # Layout using Panel
# layout = pn.Column(
#     pn.Row(pn.Column('## Geodata file viewer', dropdown)),
#     update_map
# )

# # Display the Panel dashboard
# layout.servable()